In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
from openpyxl import load_workbook
from math import log10, floor
import sys
sys._enablelegacywindowsfsencoding()


def round_to_1(x):
    return round(x, -int(floor(log10(x))))
def round_to_reference(x, y):
    return round(x, -int(floor(log10(y))))

eps = 'ε'
Q = 'Q'
G = 'Γ'
I = 'I'
delta = 'δ'
tetta = 'θ'
phi = 'φ'
H = 'Hn'
Hmax = 'Hnmax'

def get_ready_data(name):
    data = pd.read_csv(name, header=0, sep=";")
    data = data.drop(data.columns[2:22], axis=1)
    return data



def find_param(param, print_info = 0):
    param_value = []
    param_error = []
    param_total = []
    param_model = []
    i=0
    for column in data.columns:
        if (param in column and not('Δ' in column) and (not('†' in column))):
            #print data[column]
                param_value.append(data[column][0])
                param_model.append(column)
        if ('Δ'+param in column):
            #if ('Δ' in column) and (not('†' in column)):
            if (data[column][0] < 0.015) and (data[column][0] != 0):
                param_error.append(0.01)
            elif data[column][0] == 0:
                param_error.append(0.0000001)
            else:
                param_error.append(data[column][0])
        if ('†Δ'+param in column):
            #if ('Δ' in column) and (not('†' in column)):
            if (data[column][0] < 0.015) and (data[column][0] != 0):
                param_error[-1]=(0.01)
            elif data[column][0] == 0:
                param_error[-1]=(0.0000001)
            else:
                param_error[-1]=(data[column][0])

    error_r = list(map(round_to_1, param_error))
    param_r = list(map(round_to_reference, param_value, error_r))

    
    for i in range(0, len(param_r)):
        if (error_r[i] < 0.000001):
            param_r[i] = round(param_r[i], 1)
            param_total.append(str(param_r[i]))
        else:
            param_total.append(str(param_r[i])+'±'+str(error_r[i]))
            
        
    if print_info == 1:
        print (len(param_total), len(param_total)-len(param_model), param_total)
        print (param_model)
    if not param_total:
        param_total = [' ', ' ', ' ']
        return param_total
    else:
        return param_total

import os

#files = []
#for i in os.listdir("."):
#    if i.endswith('.csv'):
#        files.append(open(i))
        
#files_spc = []
#for i in os.listdir("."):
#    if i.endswith('.spc'):
#        files_spc.append(open(i))

path = r'C:/Users/nikvo/Google Drive/VostrovMossb/2018_article/Combined RT+He/relax/'
path2 = r'C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/'
path = path2

def folder(str1):
    files = []
    for i in os.listdir(path+'01 and 03/'+str1):
        if i.endswith('.csv'):
            files.append(open(path+'01 and 03/'+str1+i))
        
    files_spc = []
    for i in os.listdir(path+'01 and 03/'+str1):
        if i.endswith('.spc'):
            files_spc.append(open(path+'01 and 03/'+str1+i))
    return files, files_spc


def find_name(str_n, f_array):
    for f in f_array:
        if str_n in f.name:
            return f.name
#print(data.T.to_string())

def give_links(table, n, path, f_spc_array):
    for f in f_spc_array:
        if n[0:5] in f.name:
            table.loc[charge, '4Fe3+']['data', 'link'] = 'file:///'+path+f.name
            print (path+f.name)
 
    table.loc[charge, 'MFe3+']['data', 'link'] = 'file:///'+path+n[0:-3]+'mdl'
    table.loc[charge, '4Fe2+']['data', 'link'] = 'file:///'+path+n[0:-3]+'html'
    table.loc[charge, '4Fe3+']['data', 'data'] = 'spc'
    table.loc[charge, 'MFe3+']['data', 'data'] = 'mdl'
    table.loc[charge, '4Fe2+']['data', 'data'] = 'html'
    
def give_links_zn(table, n, path, f_spc_array):
    for f in f_spc_array:
        if n[0:5] in f.name:
            table.loc[charge, '4Fe3+']['data', 'link'] = path+f.name
 
    table.loc[charge, 'RelaxFe3+']['data', 'link'] = path+n[0:-3]+'mdl'
    table.loc[charge, '4Fe2+']['data', 'link'] = path+n[0:-3]+'html'
    table.loc[charge, '4Fe3+']['data', 'data'] = 'spc'
    table.loc[charge, 'RelaxFe3+']['data', 'data'] = 'mdl'
    table.loc[charge, '4Fe2+']['data', 'data'] = 'html'
    
#n = find_name('Co0.2_IN', files)

#file:///



In [2]:
table = pd.DataFrame(columns = ['Intensity', delta, eps, ])
oney = ['He', 'He', 'He', 'He', 'He', 'He', 'He', 'RT', 'RT', 'RT', 'RT', 'RT', 'data', 'data']
twoy = [I, delta, 'e^2qQ', H, tetta, phi, G, I, delta, eps, G, Q, 'data', 'link']
spectra = ['t1', 't2', 't3', 't4', 't5']
partial_sp = ['4Fe3+', 'MFe3+', '4Fe2+', '.', "RelaxFe2+", '..']
onex = []
twox = []
for i in range(0, len(spectra)):
    twox.append(partial_sp)
    for j in range(0, len(partial_sp)):
        onex.append(spectra[i])
twox = [j for sub in twox for j in sub]
arraysy = [oney, twoy]
arraysx = [onex, twox]
table = pd.DataFrame(columns=arraysy, index = arraysx).fillna(' ')
charge = spectra[0]
 
files, files_spc = folder('mn03t1/')
n = find_name('15036+15046', files)
data = get_ready_data(n)
give_links(table, n, path, files_spc)  


table.loc[charge, '.']['He', tetta] = 'ln(R)'
table.loc[charge, '.']['He', phi] = 'α'
table.loc[charge, '.']['He', 'e^2qQ'] = eps

    
table.loc[charge, '4Fe3+']['He', 'I'] = find_param('I1')[1]
table.loc[charge, 'MFe3+']['He', 'I'] = find_param('I2')[0]
table.loc[charge, '4Fe2+']['He', 'I'] = find_param('I3')[1]
table.loc[charge, 'RelaxFe2+']['He', 'I'] = find_param('I4')[0]
    
table.loc[charge, '4Fe3+']['He', delta] = find_param(delta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', delta] = find_param(delta+' / 2')[0]
table.loc[charge, '4Fe2+']['He', delta] = find_param(delta+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', delta] = find_param(delta+' / 4')[0]
   
table.loc[charge, '4Fe3+']['He', 'e^2qQ'] = find_param(Q+' / 1')[0]
table.loc[charge, 'MFe3+']['He', 'e^2qQ'] = find_param(Q+' / 2')[0]
table.loc[charge, '4Fe2+']['He', 'e^2qQ'] = find_param(Q+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]
    
table.loc[charge, '4Fe3+']['He', H] = find_param(H+' / 1')[0]
table.loc[charge, 'MFe3+']['He', H] = find_param(Hmax+' / 2')[0]
table.loc[charge, '4Fe2+']['He', H] = find_param(H+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', H] = find_param('H / 4')[0]
    
table.loc[charge, '4Fe3+']['He', tetta] = find_param(tetta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', tetta] = find_param(tetta+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', tetta] = find_param(tetta+' / 3')[0]

table.loc[charge, '4Fe3+']['He', phi] = find_param(phi+' / 1')[0]
table.loc[charge, 'MFe3+']['He', phi] = find_param(phi+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', phi] = find_param(phi+' / 3')[0]
   
table.loc[charge, '4Fe3+']['He', G] = find_param(G+' / 1')[0]    
table.loc[charge, 'MFe3+']['He', G] = find_param(G+' / 2')[0]
table.loc[charge, '4Fe2+']['He', G] = find_param('Γ₁ / 3 /')[0]+' '+find_param('Γ₂ / 3')[0]+' '+find_param('Γ₃ / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', G] = find_param(G+' / 4')[0]

table.loc[charge, 'RelaxFe2+']['He', tetta] = find_param('ln(R)')[0]
table.loc[charge, 'RelaxFe2+']['He', phi] = find_param('α')[3]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]



table.loc[charge, '4Fe3+']['RT', 'I'] = find_param('I5')[0]
table.loc[charge, 'MFe3+']['RT', 'I'] = find_param('I6')[0]
table.loc[charge, '4Fe2+']['RT', 'I'] = find_param('I7')[0]
table.loc[charge, 'RelaxFe2+']['RT', 'I'] = find_param('I8')[0]

table.loc[charge, '4Fe3+']['RT', delta] = find_param('delta5')[0]
table.loc[charge, 'MFe3+']['RT', delta] = find_param('delta6')[0]
table.loc[charge, '4Fe2+']['RT', delta] = find_param('delta7')[0]
table.loc[charge, 'RelaxFe2+']['RT', delta] = find_param('delta8')[0]

table.loc[charge, '4Fe3+']['RT', eps] = find_param('eps5')[0]
table.loc[charge, 'MFe3+']['RT', eps] = find_param('eps6')[0]
table.loc[charge, '4Fe2+']['RT', eps] = find_param('eps7')[0]
table.loc[charge, 'RelaxFe2+']['RT', eps] = find_param('eps8')[0]

table.loc[charge, '4Fe3+']['RT', G] = find_param('Г5')[0]
table.loc[charge, 'MFe3+']['RT', G] = find_param('Г6')[0]
table.loc[charge, '4Fe2+']['RT', G] = find_param('Г7')[0]
table.loc[charge, 'RelaxFe2+']['RT', G] = find_param('Г8')[0]



table


C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/01 and 03/mn03t1/15036+15046_1024.spc


He                                                        \
                      I          δ       e^2qQ          Hn         θ        φ   
t1 4Fe3+            0.0        0.1        -2.7       500.0      33.0     90.0   
   MFe3+       24.5±0.6  0.56±0.01  -1.76±0.01   529.0±2.0  53.0±0.2      0.0   
   4Fe2+      67.2±0.01  1.36±0.01   5.59±0.01   123.3±0.1       0.0      0.0   
   .                                         ε                 ln(R)        α   
   RelaxFe2+    8.3±0.5        1.4   1.74±0.04  80.21±0.02       0.0  0.7±0.1   
   ..                                                                           
t2 4Fe3+                                                                        
   MFe3+                                                                        
   4Fe2+                                                                        
   .                                                                            
   RelaxFe2+                                                                    
   ..                                                                           
t3 4Fe3+                                                                        
   MFe3+                                                                        
   4Fe2+                                                                        
   .                                                                            
   RelaxFe2+                                                                    
   ..                                                                           
t4 4Fe3+                                                                        
   MFe3+                                                                        
   4Fe2+                                                                        
   .                                                                            
   RelaxFe2+                                                                    
   ..                                                                           
t5 4Fe3+                                                                        
   MFe3+                                                                        
   4Fe2+                                                                        
   .                                                                            
   RelaxFe2+                                                                    
   ..                                                                           

                                                   RT                        \
                                          Γ         I          δ          ε   
t1 4Fe3+                                0.0       0.0        0.4        0.8   
   MFe3+                          0.42±0.01  22.4±0.2  0.45±0.01  0.51±0.01   
   4Fe2+      0.46±0.01 0.36±0.01 0.33±0.01  69.0±0.5  1.22±0.01  1.48±0.01   
   .                                                                          
   RelaxFe2+                            0.8   8.5±0.5  1.33±0.01  1.06±0.02   
   ..                                                                         
t2 4Fe3+                                                                      
   MFe3+                                                                      
   4Fe2+                                                                      
   .                                                                          
   RelaxFe2+                                                                  
   ..                                                                         
t3 4Fe3+                                                                      
   MFe3+                                                                      
   4Fe2+                                                                      
   .                                                                          
   Rel

In [3]:
  
charge = spectra[1]
files, files_spc = folder('mn03t2/')    
n = find_name('15048+15059', files)
data = get_ready_data(n)
give_links(table, n, path, files_spc)  


table.loc[charge, '.']['He', tetta] = 'ln(R)'
table.loc[charge, '.']['He', phi] = 'α'
table.loc[charge, '.']['He', 'e^2qQ'] = eps

    
table.loc[charge, '4Fe3+']['He', 'I'] = find_param('I1')[1]
table.loc[charge, 'MFe3+']['He', 'I'] = find_param('I2')[0]
table.loc[charge, '4Fe2+']['He', 'I'] = find_param('I3')[1]
table.loc[charge, 'RelaxFe2+']['He', 'I'] = find_param('I4')[0]
    
table.loc[charge, '4Fe3+']['He', delta] = find_param(delta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', delta] = find_param(delta+' / 2')[0]
table.loc[charge, '4Fe2+']['He', delta] = find_param(delta+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', delta] = find_param(delta+' / 4')[0]
   
table.loc[charge, '4Fe3+']['He', 'e^2qQ'] = find_param(Q+' / 1')[0]
table.loc[charge, 'MFe3+']['He', 'e^2qQ'] = find_param(Q+' / 2')[0]
table.loc[charge, '4Fe2+']['He', 'e^2qQ'] = find_param(Q+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]
    
table.loc[charge, '4Fe3+']['He', H] = find_param(H+' / 1')[0]
table.loc[charge, 'MFe3+']['He', H] = find_param(Hmax+' / 2')[0]
table.loc[charge, '4Fe2+']['He', H] = find_param(H+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', H] = find_param('H / 4')[0]
    
table.loc[charge, '4Fe3+']['He', tetta] = find_param(tetta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', tetta] = find_param(tetta+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', tetta] = find_param(tetta+' / 3')[0]

table.loc[charge, '4Fe3+']['He', phi] = find_param(phi+' / 1')[0]
table.loc[charge, 'MFe3+']['He', phi] = find_param(phi+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', phi] = find_param(phi+' / 3')[0]
   
table.loc[charge, '4Fe3+']['He', G] = find_param(G+' / 1')[0]    
table.loc[charge, 'MFe3+']['He', G] = find_param(G+' / 2')[0]
table.loc[charge, '4Fe2+']['He', G] = find_param('Γ₁ / 3 /')[0]+' '+find_param('Γ₂ / 3')[0]+' '+find_param('Γ₃ / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', G] = find_param(G+' / 4')[0]

table.loc[charge, 'RelaxFe2+']['He', tetta] = find_param('ln(R)')[0]
table.loc[charge, 'RelaxFe2+']['He', phi] = find_param('α')[3]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]



table.loc[charge, '4Fe3+']['RT', 'I'] = find_param('I5')[0]
table.loc[charge, 'MFe3+']['RT', 'I'] = find_param('I6')[0]
table.loc[charge, '4Fe2+']['RT', 'I'] = find_param('I7')[0]
table.loc[charge, 'RelaxFe2+']['RT', 'I'] = find_param('I8')[0]

table.loc[charge, '4Fe3+']['RT', delta] = find_param('delta5')[0]
table.loc[charge, 'MFe3+']['RT', delta] = find_param('delta6')[0]
table.loc[charge, '4Fe2+']['RT', delta] = find_param('delta7')[0]
table.loc[charge, 'RelaxFe2+']['RT', delta] = find_param('delta8')[0]

table.loc[charge, '4Fe3+']['RT', eps] = find_param('eps5')[0]
table.loc[charge, 'MFe3+']['RT', eps] = find_param('eps6')[0]
table.loc[charge, '4Fe2+']['RT', eps] = find_param('eps7')[0]
table.loc[charge, 'RelaxFe2+']['RT', eps] = find_param('eps8')[0]

table.loc[charge, '4Fe3+']['RT', G] = find_param('Г5')[0]
table.loc[charge, 'MFe3+']['RT', G] = find_param('Г6')[0]
table.loc[charge, '4Fe2+']['RT', G] = find_param('Г7')[0]
table.loc[charge, 'RelaxFe2+']['RT', G] = find_param('Г8')[0]



table


C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/01 and 03/mn03t2/15048+15059_1024.spc


He                                                 \
                       I          δ       e^2qQ          Hn           θ   
t1 4Fe3+             0.0        0.1        -2.7       500.0        33.0   
   MFe3+        24.5±0.6  0.56±0.01  -1.76±0.01   529.0±2.0    53.0±0.2   
   4Fe2+       67.2±0.01  1.36±0.01   5.59±0.01   123.3±0.1         0.0   
   .                                          ε                   ln(R)   
   RelaxFe2+     8.3±0.5        1.4   1.74±0.04  80.21±0.02         0.0   
   ..                                                                     
t2 4Fe3+             0.0        0.1        -2.7       500.0        33.0   
   MFe3+        58.7±0.4  0.56±0.01  -1.92±0.01   526.0±1.0  60.81±0.08   
   4Fe2+      25.22±0.01  1.36±0.01   5.55±0.03   117.1±0.8         0.0   
   .                                          ε                   ln(R)   
   RelaxFe2+    16.1±0.5        1.4   1.68±0.02   82.9±0.03         0.0   
   ..                                                                     
t3 4Fe3+                                                                  
   MFe3+                                                                  
   4Fe2+                                                                  
   .                                                                      
   RelaxFe2+                                                              
   ..                                                                     
t4 4Fe3+                                                                  
   MFe3+                                                                  
   4Fe2+                                                                  
   .                                                                      
   RelaxFe2+                                                              
   ..                                                                     
t5 4Fe3+                                                                  
   MFe3+                                                                  
   4Fe2+                                                                  
   .                                                                      
   RelaxFe2+                                                              
   ..                                                                     

                                                              RT             \
                      φ                              Γ         I          δ   
t1 4Fe3+           90.0                            0.0       0.0        0.4   
   MFe3+            0.0                      0.42±0.01  22.4±0.2  0.45±0.01   
   4Fe2+            0.0  0.46±0.01 0.36±0.01 0.33±0.01  69.0±0.5  1.22±0.01   
   .                  α                                                       
   RelaxFe2+    0.7±0.1                            0.8   8.5±0.5  1.33±0.01   
   ..                                                                         
t2 4Fe3+           90.0                            0.0       0.0        0.4   
   MFe3+       0.0±0.01                      0.42±0.01  57.7±0.3  0.45±0.01   
   4Fe2+            0.0   0.78±0.04 0.58±0.03 0.4±0.02  25.8±0.5  1.21±0.01   
   .                  α                                                       
   RelaxFe2+  0.33±0.06                            0.8  16.4±0.6  1.29±0.01   
   ..                                                                         
t3 4Fe3+                                                                      
   MFe3+                                                                      
   4Fe2+                                                                      
   .                                                                          
   RelaxFe2+                                                                  
   ..                                                                         
t4 4Fe3+                                 

In [4]:
  
charge = spectra[2]
files, files_spc = folder('mn03t3/')
n = find_name('15023+15019', files)
data = get_ready_data(n)
give_links(table, n, path, files_spc)  

table.loc[charge, '.']['He', tetta] = 'ln(R)'
table.loc[charge, '.']['He', phi] = 'α'
table.loc[charge, '.']['He', 'e^2qQ'] = eps

    
table.loc[charge, '4Fe3+']['He', 'I'] = find_param('I1')[1]
table.loc[charge, 'MFe3+']['He', 'I'] = find_param('I2')[0]
table.loc[charge, '4Fe2+']['He', 'I'] = find_param('I3')[1]
table.loc[charge, 'RelaxFe2+']['He', 'I'] = find_param('I4')[0]
    
table.loc[charge, '4Fe3+']['He', delta] = find_param(delta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', delta] = find_param(delta+' / 2')[0]
table.loc[charge, '4Fe2+']['He', delta] = find_param(delta+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', delta] = find_param(delta+' / 4')[0]
   
table.loc[charge, '4Fe3+']['He', 'e^2qQ'] = find_param(Q+' / 1')[0]
table.loc[charge, 'MFe3+']['He', 'e^2qQ'] = find_param(Q+' / 2')[0]
table.loc[charge, '4Fe2+']['He', 'e^2qQ'] = find_param(Q+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]
    
table.loc[charge, '4Fe3+']['He', H] = find_param(H+' / 1')[0]
table.loc[charge, 'MFe3+']['He', H] = find_param(Hmax+' / 2')[0]
table.loc[charge, '4Fe2+']['He', H] = find_param(H+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', H] = find_param('H / 4')[0]
    
table.loc[charge, '4Fe3+']['He', tetta] = find_param(tetta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', tetta] = find_param(tetta+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', tetta] = find_param(tetta+' / 3')[0]

table.loc[charge, '4Fe3+']['He', phi] = find_param(phi+' / 1')[0]
table.loc[charge, 'MFe3+']['He', phi] = find_param(phi+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', phi] = find_param(phi+' / 3')[0]
   
table.loc[charge, '4Fe3+']['He', G] = find_param(G+' / 1')[0]    
table.loc[charge, 'MFe3+']['He', G] = find_param(G+' / 2')[0]
table.loc[charge, '4Fe2+']['He', G] = find_param('Γ₁ / 3 /')[0]+' '+find_param('Γ₂ / 3')[0]+' '+find_param('Γ₃ / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', G] = find_param(G+' / 4')[0]

table.loc[charge, 'RelaxFe2+']['He', tetta] = find_param('ln(R)')[0]
table.loc[charge, 'RelaxFe2+']['He', phi] = find_param('α')[3]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]



table.loc[charge, '4Fe3+']['RT', 'I'] = find_param('I5')[0]
table.loc[charge, 'MFe3+']['RT', 'I'] = find_param('I6')[0]
table.loc[charge, '4Fe2+']['RT', 'I'] = find_param('I7')[0]
table.loc[charge, 'RelaxFe2+']['RT', 'I'] = find_param('I8')[0]

table.loc[charge, '4Fe3+']['RT', delta] = find_param('delta5')[0]
table.loc[charge, 'MFe3+']['RT', delta] = find_param('delta6')[0]
table.loc[charge, '4Fe2+']['RT', delta] = find_param('delta7')[0]
table.loc[charge, 'RelaxFe2+']['RT', delta] = find_param('delta8')[0]

table.loc[charge, '4Fe3+']['RT', eps] = find_param('eps5')[0]
table.loc[charge, 'MFe3+']['RT', eps] = find_param('eps6')[0]
table.loc[charge, '4Fe2+']['RT', eps] = find_param('eps7')[0]
table.loc[charge, 'RelaxFe2+']['RT', eps] = find_param('eps8')[0]

table.loc[charge, '4Fe3+']['RT', G] = find_param('Г5')[0]
table.loc[charge, 'MFe3+']['RT', G] = find_param('Г6')[0]
table.loc[charge, '4Fe2+']['RT', G] = find_param('Г7')[0]
table.loc[charge, 'RelaxFe2+']['RT', G] = find_param('Г8')[0]



table

C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/01 and 03/mn03t3/15023+15019_1024.spc


He                                                 \
                       I          δ       e^2qQ          Hn           θ   
t1 4Fe3+             0.0        0.1        -2.7       500.0        33.0   
   MFe3+        24.5±0.6  0.56±0.01  -1.76±0.01   529.0±2.0    53.0±0.2   
   4Fe2+       67.2±0.01  1.36±0.01   5.59±0.01   123.3±0.1         0.0   
   .                                          ε                   ln(R)   
   RelaxFe2+     8.3±0.5        1.4   1.74±0.04  80.21±0.02         0.0   
   ..                                                                     
t2 4Fe3+             0.0        0.1        -2.7       500.0        33.0   
   MFe3+        58.7±0.4  0.56±0.01  -1.92±0.01   526.0±1.0  60.81±0.08   
   4Fe2+      25.22±0.01  1.36±0.01   5.55±0.03   117.1±0.8         0.0   
   .                                          ε                   ln(R)   
   RelaxFe2+    16.1±0.5        1.4   1.68±0.02   82.9±0.03         0.0   
   ..                                                                     
t3 4Fe3+             0.0        0.6        -2.7       500.0        33.0   
   MFe3+        69.1±0.5  0.56±0.01  -2.01±0.01   527.0±1.0   62.9±0.06   
   4Fe2+       8.86±0.01        1.4     5.6±0.1       120.0         0.0   
   .                                          ε                   ln(R)   
   RelaxFe2+    22.1±0.6        1.4   1.69±0.02    87.0±5.0         0.0   
   ..                                                                     
t4 4Fe3+                                                                  
   MFe3+                                                                  
   4Fe2+                                                                  
   .                                                                      
   RelaxFe2+                                                              
   ..                                                                     
t5 4Fe3+                                                                  
   MFe3+                                                                  
   4Fe2+                                                                  
   .                                                                      
   RelaxFe2+                                                              
   ..                                                                     

                                                              RT             \
                      φ                              Γ         I          δ   
t1 4Fe3+           90.0                            0.0       0.0        0.4   
   MFe3+            0.0                      0.42±0.01  22.4±0.2  0.45±0.01   
   4Fe2+            0.0  0.46±0.01 0.36±0.01 0.33±0.01  69.0±0.5  1.22±0.01   
   .                  α                                                       
   RelaxFe2+    0.7±0.1                            0.8   8.5±0.5  1.33±0.01   
   ..                                                                         
t2 4Fe3+           90.0                            0.0       0.0        0.4   
   MFe3+       0.0±0.01                      0.42±0.01  57.7±0.3  0.45±0.01   
   4Fe2+            0.0   0.78±0.04 0.58±0.03 0.4±0.02  25.8±0.5  1.21±0.01   
   .                  α                                                       
   RelaxFe2+  0.33±0.06                            0.8  16.4±0.6  1.29±0.01   
   ..                                                                         
t3 4Fe3+           90.0                            0.3       0.0        0.4   
   MFe3+           -0.0                      0.45±0.01  68.9±0.8  0.43±0.01   
   4Fe2+            0.0                    0.7 0.7 0.7   8.9±0.7   1.2±0.01   
   .                  α                                                       
   RelaxFe2+   0.8±0.04                            0.8  22.0±1.0  1.28±0.01   
   ..                                                                         
t4 4Fe3+                                 

In [5]:
  
charge = spectra[3]
files, files_spc = folder('mn03t4/')    
n = find_name('15030+33+15037', files)
data = get_ready_data(n)
give_links(table, n, path, files_spc)  

table.loc[charge, '.']['He', tetta] = 'ln(R)'
table.loc[charge, '.']['He', phi] = 'α'
table.loc[charge, '.']['He', 'e^2qQ'] = eps

    
table.loc[charge, '4Fe3+']['He', 'I'] = find_param('I1')[1]
table.loc[charge, 'MFe3+']['He', 'I'] = find_param('I2')[0]
table.loc[charge, '4Fe2+']['He', 'I'] = find_param('I3')[1]
table.loc[charge, 'RelaxFe2+']['He', 'I'] = find_param('I4')[0]
    
table.loc[charge, '4Fe3+']['He', delta] = find_param(delta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', delta] = find_param(delta+' / 2')[0]
table.loc[charge, '4Fe2+']['He', delta] = find_param(delta+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', delta] = find_param(delta+' / 4')[0]
   
table.loc[charge, '4Fe3+']['He', 'e^2qQ'] = find_param(Q+' / 1')[0]
table.loc[charge, 'MFe3+']['He', 'e^2qQ'] = find_param(Q+' / 2')[0]
table.loc[charge, '4Fe2+']['He', 'e^2qQ'] = find_param(Q+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]
    
table.loc[charge, '4Fe3+']['He', H] = find_param(H+' / 1')[0]
table.loc[charge, 'MFe3+']['He', H] = find_param(Hmax+' / 2')[0]
table.loc[charge, '4Fe2+']['He', H] = find_param(H+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', H] = find_param('H / 4')[0]
    
table.loc[charge, '4Fe3+']['He', tetta] = find_param(tetta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', tetta] = find_param(tetta+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', tetta] = find_param(tetta+' / 3')[0]

table.loc[charge, '4Fe3+']['He', phi] = find_param(phi+' / 1')[0]
table.loc[charge, 'MFe3+']['He', phi] = find_param(phi+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', phi] = find_param(phi+' / 3')[0]
   
table.loc[charge, '4Fe3+']['He', G] = find_param(G+' / 1')[0]    
table.loc[charge, 'MFe3+']['He', G] = find_param(G+' / 2')[0]
table.loc[charge, '4Fe2+']['He', G] = find_param('Γ₁ / 3 /')[0]+' '+find_param('Γ₂ / 3')[0]+' '+find_param('Γ₃ / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', G] = find_param(G+' / 4')[0]

table.loc[charge, 'RelaxFe2+']['He', tetta] = find_param('ln(R)')[0]
table.loc[charge, 'RelaxFe2+']['He', phi] = find_param('α')[3]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]



table.loc[charge, '4Fe3+']['RT', 'I'] = find_param('I5')[0]
table.loc[charge, 'MFe3+']['RT', 'I'] = find_param('I6')[0]
table.loc[charge, '4Fe2+']['RT', 'I'] = find_param('I7')[0]
table.loc[charge, 'RelaxFe2+']['RT', 'I'] = find_param('I8')[0]

table.loc[charge, '4Fe3+']['RT', delta] = find_param('delta5')[0]
table.loc[charge, 'MFe3+']['RT', delta] = find_param('delta6')[0]
table.loc[charge, '4Fe2+']['RT', delta] = find_param('delta7')[0]
table.loc[charge, 'RelaxFe2+']['RT', delta] = find_param('delta8')[0]

table.loc[charge, '4Fe3+']['RT', eps] = find_param('eps5')[0]
table.loc[charge, 'MFe3+']['RT', eps] = find_param('eps6')[0]
table.loc[charge, '4Fe2+']['RT', eps] = find_param('eps7')[0]
table.loc[charge, 'RelaxFe2+']['RT', eps] = find_param('eps8')[0]

table.loc[charge, '4Fe3+']['RT', G] = find_param('Г5')[0]
table.loc[charge, 'MFe3+']['RT', G] = find_param('Г6')[0]
table.loc[charge, '4Fe2+']['RT', G] = find_param('Г7')[0]
table.loc[charge, 'RelaxFe2+']['RT', G] = find_param('Г8')[0]



table

C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/01 and 03/mn03t4/15030+15037_1024-D1.spc
C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/01 and 03/mn03t4/15030+33+15037_1024.spc


He                                                 \
                       I          δ       e^2qQ          Hn           θ   
t1 4Fe3+             0.0        0.1        -2.7       500.0        33.0   
   MFe3+        24.5±0.6  0.56±0.01  -1.76±0.01   529.0±2.0    53.0±0.2   
   4Fe2+       67.2±0.01  1.36±0.01   5.59±0.01   123.3±0.1         0.0   
   .                                          ε                   ln(R)   
   RelaxFe2+     8.3±0.5        1.4   1.74±0.04  80.21±0.02         0.0   
   ..                                                                     
t2 4Fe3+             0.0        0.1        -2.7       500.0        33.0   
   MFe3+        58.7±0.4  0.56±0.01  -1.92±0.01   526.0±1.0  60.81±0.08   
   4Fe2+      25.22±0.01  1.36±0.01   5.55±0.03   117.1±0.8         0.0   
   .                                          ε                   ln(R)   
   RelaxFe2+    16.1±0.5        1.4   1.68±0.02   82.9±0.03         0.0   
   ..                                                                     
t3 4Fe3+             0.0        0.6        -2.7       500.0        33.0   
   MFe3+        69.1±0.5  0.56±0.01  -2.01±0.01   527.0±1.0   62.9±0.06   
   4Fe2+       8.86±0.01        1.4     5.6±0.1       120.0         0.0   
   .                                          ε                   ln(R)   
   RelaxFe2+    22.1±0.6        1.4   1.69±0.02    87.0±5.0         0.0   
   ..                                                                     
t4 4Fe3+      18.86±0.01  0.58±0.01  -2.49±0.01   497.9±0.7        34.3   
   MFe3+        71.9±0.9  0.55±0.01  -2.13±0.01   508.6±0.6    35.0±2.0   
   4Fe2+       3.32±0.01        1.4         5.6       120.0         0.0   
   .                                          ε                   ln(R)   
   RelaxFe2+     6.0±0.4        1.4   1.75±0.06        88.0         0.0   
   ..                                                                     
t5 4Fe3+                                                                  
   MFe3+                                                                  
   4Fe2+                                                                  
   .                                                                      
   RelaxFe2+                                                              
   ..                                                                     

                                                              RT             \
                      φ                              Γ         I          δ   
t1 4Fe3+           90.0                            0.0       0.0        0.4   
   MFe3+            0.0                      0.42±0.01  22.4±0.2  0.45±0.01   
   4Fe2+            0.0  0.46±0.01 0.36±0.01 0.33±0.01  69.0±0.5  1.22±0.01   
   .                  α                                                       
   RelaxFe2+    0.7±0.1                            0.8   8.5±0.5  1.33±0.01   
   ..                                                                         
t2 4Fe3+           90.0                            0.0       0.0        0.4   
   MFe3+       0.0±0.01                      0.42±0.01  57.7±0.3  0.45±0.01   
   4Fe2+            0.0   0.78±0.04 0.58±0.03 0.4±0.02  25.8±0.5  1.21±0.01   
   .                  α                                                       
   RelaxFe2+  0.33±0.06                            0.8  16.4±0.6  1.29±0.01   
   ..                                                                         
t3 4Fe3+           90.0                            0.3       0.0        0.4   
   MFe3+           -0.0                      0.45±0.01  68.9±0.8  0.43±0.01   
   4Fe2+            0.0                    0.7 0.7 0.7   8.9±0.7   1.2±0.01   
   .                  α                                                       
   RelaxFe2+   0.8±0.04                            0.8  22.0±1.0  1.28±0.01   
   ..                                                                         
t4 4Fe3+           72.2                  

In [6]:
  
charge = spectra[4]
files, files_spc = folder('mn03t5/')   
n = find_name('15016+15014', files)
data = get_ready_data(n)
give_links(table, n, path, files_spc)  

table.loc[charge, '.']['He', tetta] = 'ln(R)'
table.loc[charge, '.']['He', phi] = 'α'
table.loc[charge, '.']['He', 'e^2qQ'] = eps

    
table.loc[charge, '4Fe3+']['He', 'I'] = find_param('I1')[1]
table.loc[charge, 'MFe3+']['He', 'I'] = find_param('I2')[0]
table.loc[charge, '4Fe2+']['He', 'I'] = find_param('I3')[1]
table.loc[charge, 'RelaxFe2+']['He', 'I'] = find_param('I4')[0]
    
table.loc[charge, '4Fe3+']['He', delta] = find_param(delta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', delta] = find_param(delta+' / 2')[0]
table.loc[charge, '4Fe2+']['He', delta] = find_param(delta+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', delta] = find_param(delta+' / 4')[0]
   
table.loc[charge, '4Fe3+']['He', 'e^2qQ'] = find_param(Q+' / 1')[0]
table.loc[charge, 'MFe3+']['He', 'e^2qQ'] = find_param(Q+' / 2')[0]
table.loc[charge, '4Fe2+']['He', 'e^2qQ'] = find_param(Q+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]
    
table.loc[charge, '4Fe3+']['He', H] = find_param(H+' / 1')[0]
table.loc[charge, 'MFe3+']['He', H] = find_param(Hmax+' / 2')[0]
table.loc[charge, '4Fe2+']['He', H] = find_param(H+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', H] = find_param('H / 4')[0]
    
table.loc[charge, '4Fe3+']['He', tetta] = find_param(tetta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', tetta] = find_param(tetta+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', tetta] = find_param(tetta+' / 3')[0]

table.loc[charge, '4Fe3+']['He', phi] = find_param(phi+' / 1')[0]
table.loc[charge, 'MFe3+']['He', phi] = find_param(phi+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', phi] = find_param(phi+' / 3')[0]
   
table.loc[charge, '4Fe3+']['He', G] = find_param(G+' / 1')[0]    
table.loc[charge, 'MFe3+']['He', G] = find_param(G+' / 2')[0]
table.loc[charge, '4Fe2+']['He', G] = find_param('Γ₁ / 3 /')[0]+' '+find_param('Γ₂ / 3')[0]+' '+find_param('Γ₃ / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', G] = find_param(G+' / 4')[0]

table.loc[charge, 'RelaxFe2+']['He', tetta] = find_param('ln(R)')[0]
table.loc[charge, 'RelaxFe2+']['He', phi] = find_param('α')[3]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]



table.loc[charge, '4Fe3+']['RT', 'I'] = find_param('I5')[0]
table.loc[charge, 'MFe3+']['RT', 'I'] = find_param('I6')[0]
table.loc[charge, '4Fe2+']['RT', 'I'] = find_param('I7')[0]
table.loc[charge, 'RelaxFe2+']['RT', 'I'] = find_param('I8')[0]

table.loc[charge, '4Fe3+']['RT', delta] = find_param('delta5')[0]
table.loc[charge, 'MFe3+']['RT', delta] = find_param('delta6')[0]
table.loc[charge, '4Fe2+']['RT', delta] = find_param('delta7')[0]
table.loc[charge, 'RelaxFe2+']['RT', delta] = find_param('delta8')[0]

table.loc[charge, '4Fe3+']['RT', eps] = find_param('eps5')[0]
table.loc[charge, 'MFe3+']['RT', eps] = find_param('eps6')[0]
table.loc[charge, '4Fe2+']['RT', eps] = find_param('eps7')[0]
table.loc[charge, 'RelaxFe2+']['RT', eps] = find_param('eps8')[0]

table.loc[charge, '4Fe3+']['RT', G] = find_param('Г5')[0]
table.loc[charge, 'MFe3+']['RT', G] = find_param('Г6')[0]
table.loc[charge, '4Fe2+']['RT', G] = find_param('Г7')[0]
table.loc[charge, 'RelaxFe2+']['RT', G] = find_param('Г8')[0]

C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/01 and 03/mn03t5/15016+15014_1024.spc


In [7]:
table

He                                                 \
                       I          δ       e^2qQ          Hn           θ   
t1 4Fe3+             0.0        0.1        -2.7       500.0        33.0   
   MFe3+        24.5±0.6  0.56±0.01  -1.76±0.01   529.0±2.0    53.0±0.2   
   4Fe2+       67.2±0.01  1.36±0.01   5.59±0.01   123.3±0.1         0.0   
   .                                          ε                   ln(R)   
   RelaxFe2+     8.3±0.5        1.4   1.74±0.04  80.21±0.02         0.0   
   ..                                                                     
t2 4Fe3+             0.0        0.1        -2.7       500.0        33.0   
   MFe3+        58.7±0.4  0.56±0.01  -1.92±0.01   526.0±1.0  60.81±0.08   
   4Fe2+      25.22±0.01  1.36±0.01   5.55±0.03   117.1±0.8         0.0   
   .                                          ε                   ln(R)   
   RelaxFe2+    16.1±0.5        1.4   1.68±0.02   82.9±0.03         0.0   
   ..                                                                     
t3 4Fe3+             0.0        0.6        -2.7       500.0        33.0   
   MFe3+        69.1±0.5  0.56±0.01  -2.01±0.01   527.0±1.0   62.9±0.06   
   4Fe2+       8.86±0.01        1.4     5.6±0.1       120.0         0.0   
   .                                          ε                   ln(R)   
   RelaxFe2+    22.1±0.6        1.4   1.69±0.02    87.0±5.0         0.0   
   ..                                                                     
t4 4Fe3+      18.86±0.01  0.58±0.01  -2.49±0.01   497.9±0.7        34.3   
   MFe3+        71.9±0.9  0.55±0.01  -2.13±0.01   508.6±0.6    35.0±2.0   
   4Fe2+       3.32±0.01        1.4         5.6       120.0         0.0   
   .                                          ε                   ln(R)   
   RelaxFe2+     6.0±0.4        1.4   1.75±0.06        88.0         0.0   
   ..                                                                     
t5 4Fe3+      44.14±0.01  0.56±0.01  -2.61±0.01   496.3±0.6        33.0   
   MFe3+        48.0±3.0  0.56±0.01  -2.05±0.02   508.4±0.5    33.0±1.0   
   4Fe2+       2.82±0.01        1.4         5.5       120.0         0.0   
   .                                          ε                   ln(R)   
   RelaxFe2+     5.0±1.0        1.4   1.64±0.07        88.0         0.0   
   ..                                                                     

                                                              RT             \
                      φ                              Γ         I          δ   
t1 4Fe3+           90.0                            0.0       0.0        0.4   
   MFe3+            0.0                      0.42±0.01  22.4±0.2  0.45±0.01   
   4Fe2+            0.0  0.46±0.01 0.36±0.01 0.33±0.01  69.0±0.5  1.22±0.01   
   .                  α                                                       
   RelaxFe2+    0.7±0.1                            0.8   8.5±0.5  1.33±0.01   
   ..                                                                         
t2 4Fe3+           90.0                            0.0       0.0        0.4   
   MFe3+       0.0±0.01                      0.42±0.01  57.7±0.3  0.45±0.01   
   4Fe2+            0.0   0.78±0.04 0.58±0.03 0.4±0.02  25.8±0.5  1.21±0.01   
   .                  α                                                       
   RelaxFe2+  0.33±0.06                            0.8  16.4±0.6  1.29±0.01   
   ..                                                                         
t3 4Fe3+           90.0                            0.3       0.0        0.4   
   MFe3+           -0.0                      0.45±0.01  68.9±0.8  0.43±0.01   
   4Fe2+            0.0                    0.7 0.7 0.7   8.9±0.7   1.2±0.01   
   .                  α                                                       
   RelaxFe2+   0.8±0.04                            0.8  22.0±1.0  1.28±0.01   
   ..                                                                         
t4 4Fe3+           72.2                  

In [8]:
writer = pd.ExcelWriter(r"tableMn03.xlsx", engine = 'openpyxl')
table.to_excel(writer, sheet_name='Mn')
writer.save()
#writer.close()